In [104]:
import numpy as np
import random
from scipy.spatial import distance

n_dimensions=3
n_points=100

#Create samples on a unit sphere in desired dimensions
def hypersphere(npoints, ndim):
    np.random.seed(1)
    vec=np.random.randn(ndim,npoints) #creates a random sample from a Gaussian distribution in the form of an array of dimensions: ndim x npoints
    vec/=np.linalg.norm(vec,axis=0) #divides each vector by its norm, which turns each vector into a unit vector (length 1). Here we obtain samples from the unit sphere in the dimension we stated in the beginning of the function.
    return(vec)

In [81]:
#Randomly select a point on the hypersphere to be the center of the ball
random.seed(1)
hsphere=hypersphere(n_points,n_dimensions)
dim_array=np.shape(hsphere) #defines the shape of the array
rng=random.randint(0,dim_array[1])#pulls random number from 0 to n_points and assign to rng
center_ball=hsphere[:,rng] #uses randomly generated number to extract one row of points
print(center_ball)
print(hsphere[0:2,0])

[-0.46457661 -0.85509238 -0.23018598]
[ 0.93795301 -0.25818745]


In [157]:
radii = np.arange(.1,1.01,.01) #creates an array of radii values to iterate through

#Find the points on the hypersphere within each radii
for i in radii: #iteratest through each radii value
    X=np.empty([0,dim_array[0]]) #creates an empty array to be filled in with the values on the hypersphere that fall within the ball radii
    for j in np.arange(0,n_points): #iterates through array of index values for each point
        if distance.euclidean(hsphere[:,j],center_ball)<=i: #selects points whose distance to center is shorter than the length of the radius
            if np.shape(X)==(0,dim_array[0]):
                X=np.append(X,hsphere[:,j]) #appends first point coordinates to the empty array
            else:
                X=np.vstack((X,hsphere[:,j])) #vertically stacks subsequent points to the empty array
    dim_X=np.shape(X) #saves dimensions of matrix for points within the current radius
    #Need to set up covariance matrix within this and perform PCA

In [113]:
distance.euclidean(hsphere[:,22],center_ball)

1.921649874320172

In [150]:
x=np.empty([0,3])

In [154]:
a=[1,2,3]
x=np.vstack((x,a))
x

array([[1., 2., 3.],
       [1., 2., 3.]])

In [151]:
np.shape(x)

(0, 3)